# Support Vector Machine Regression 

## Mục tiêu:
- Xây dựng mô hình SVM Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 

## Giới thiệu

SVR là phiên bản hồi quy của SVM, có khả năng mô hình hóa quan hệ phi tuyến bằng kernel.

### **Ưu điểm:**

- Mạnh mẽ trong dữ liệu phi tuyến.

- Không bị ảnh hưởng nhiều bởi outliers.

- Tổng quát hóa tốt.

### **Nhược điểm:**

- Chậm khi dữ liệu lớn.

- Cần chuẩn hóa dữ liệu.

- Khó chọn tham số (C, epsilon, gamma).

## Bước 1 - Import các thư viện cần thiết

1.1. Import thư viện

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

1.2. Cấu hình thư mục

In [14]:
RANDOM_STATE = 42
os.makedirs("../model/2_SVM_regression", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý 

In [15]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [16]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. poverty_ratio
  4. pop_growth
  5. gdp_per_capita
  6. gdp_growth
  7. sanitation
  8. electricity
  9. water_access
  10. co2_emissions
  11. slum_population
  12. labor_force

Kích thước X_train: (3255, 12)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình LinearSVR bằng GridSearchCV (k=5)


4.1. Tìm cấu hình tốt nhất cho LinearSVR bằng 5-fold CV trên tập train

In [17]:
# Lưới tham số cho LinearSVR
param_grid_linsvr = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],
    'epsilon': [0.0, 0.1, 0.2, 0.5],
    'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
    "fit_intercept": [True],
    "tol": [1e-4, 1e-3],
    "max_iter": [1000, 3000, 5000]
}

print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid_linsvr.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative MSE (neg_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("="*60)

# Tìm ra mô hình LinearSVR tốt nhất
linsvr = LinearSVR(random_state=RANDOM_STATE, dual='auto')
cv_linsvr = GridSearchCV(linsvr, param_grid_linsvr, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
cv_linsvr.fit(X_train, y_train)

best_linsvr = cv_linsvr.best_estimator_
print("Tham số tốt nhất cho mô hình LinearSVR:", cv_linsvr.best_params_)

Số lượng tổ hợp tham số: 240
Phương pháp: GridSearchCV với 5-Fold Cross-Validation
Metric đánh giá: Negative MSE (neg_mean_squared_error)

Đang thực hiện tìm kiếm...
Fitting 5 folds for each of 240 candidates, totalling 1200 fits
Tham số tốt nhất cho mô hình LinearSVR: {'C': 0.01, 'epsilon': 0.0, 'fit_intercept': True, 'loss': 'squared_epsilon_insensitive', 'max_iter': 1000, 'tol': 0.0001}


4.2. Đánh giá và lưu model

In [18]:
# Đánh giá trên tập train
y_pred_linsvr = best_linsvr.predict(X_train)
linsvr_rmse = np.sqrt(mean_squared_error(y_train, y_pred_linsvr))
linsvr_r2  = r2_score(y_train, y_pred_linsvr)
linsvr_mae = mean_absolute_error(y_train, y_pred_linsvr)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: LinearSVR:")
print(f"RMSE loss: {linsvr_rmse:.3f}")
print(f"MAE loss: {linsvr_mae:.3f}")
print(f"R2 loss: {linsvr_r2:.3f}")

# Lưu model
joblib.dump(best_linsvr, "../model/2_SVM_regression/linear_svr_best.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: LinearSVR:
RMSE loss: 4.485
MAE loss: 3.361
R2 loss: 0.733


['../model/2_SVM_regression/linear_svr_best.pkl']

## Bước 5 - Xây dựng và huấn luyện mô hình SVR (kernel=rbf) bằng GridSearchCV (k=5)

5.1. Tìm cấu hình tốt nhất cho LinearSVR bằng 5-fold CV trên tập train

In [19]:
# Lưới tham số cho SVR (RBF)
param_grid_svr = {
    'C': [0.1, 1, 10, 100, 1000],
    'epsilon': [0.001, 0.01, 0.1, 0.5, 1.0],
    'gamma': ["scale", "auto", 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}

print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid_svr.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative MSE (neg_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("="*60)

svr = SVR()
cv_svr = GridSearchCV(svr, param_grid_svr, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
cv_svr.fit(X_train, y_train)
best_svr = cv_svr.best_estimator_
print("Tham số tốt nhất cho mô hình SVR:", cv_svr.best_params_)

Số lượng tổ hợp tham số: 125
Phương pháp: GridSearchCV với 5-Fold Cross-Validation
Metric đánh giá: Negative MSE (neg_mean_squared_error)

Đang thực hiện tìm kiếm...
Fitting 5 folds for each of 125 candidates, totalling 625 fits
Tham số tốt nhất cho mô hình SVR: {'C': 100, 'epsilon': 0.01, 'gamma': 'auto', 'kernel': 'rbf'}


5.2. Đánh giá và lưu model

In [20]:
# Đánh giá trên TRAIN
y_pred_svr = best_svr.predict(X_train)
svr_rmse = np.sqrt(mean_squared_error(y_train, y_pred_svr))
svr_r2  = r2_score(y_train, y_pred_svr)
svr_mae = mean_absolute_error(y_train, y_pred_svr)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: SVR:")
print(f"RMSE loss: {svr_rmse:.3f}")
print(f"MAE loss: {svr_mae:.3f}")
print(f"R2 loss: {svr_r2:.3f}")

# Lưu model
joblib.dump(best_svr, "../model/2_SVM_regression/svr_rbf_best.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: SVR:
RMSE loss: 1.763
MAE loss: 0.808
R2 loss: 0.959


['../model/2_SVM_regression/svr_rbf_best.pkl']

## Kết luận
Thông qua notebook này, ta đã:
- Tìm được bộ tham số tốt nhất tìm được cho LinearSVR/SVR-RBF
- Train các mô hình này trên tập dữ liệu huấn luyện
- Đánh giá các mô hình thông qua các loại loss khác nhau: MAE, RMSE, R2